In [0]:
from pyspark.sql.functions import col, when
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#Ler tabela
df = spark.read.table("portfolio_credit.german_credit_bronze")

# Vamos criar uma visualização temporária no Pandas para facilitar os gráficos
# Como o dado é pequeno - 1000 linhas - podemos converter para Pandas sem travar a memória
pdf = df.toPandas()

print("Dados carregados para análise!")
display(pdf.head())

In [0]:
# Célula 2: Decodificação (Data Enrichment)

# Dicionários baseados na documentação oficial do Dataset
decode_map = {
    "checking_status": {
        'A11': '< 0 DM', 
        'A12': '0 - 200 DM', 
        'A13': '> 200 DM', 
        'A14': 'Sem Conta Corrente'
    },
    "credit_history": {
        'A30': 'Sem créditos/todos pagos',
        'A31': 'Todos créditos pagos em dia',
        'A32': 'Créditos existentes pagos em dia',
        'A33': 'Pagamentos atrasados no passado',
        'A34': 'Conta crítica/outros créditos existentes'
    },
    "purpose": {
        'A40': 'Carro (Novo)', 'A41': 'Carro (Usado)', 
        'A42': 'Mobília/Equipamento', 'A43': 'Rádio/TV', 
        'A44': 'Eletrodomésticos', 'A45': 'Reparos', 
        'A46': 'Educação', 'A47': 'Férias', 
        'A48': 'Retreinamento', 'A49': 'Negócios', 'A410': 'Outros'
    },
    "savings_status": {
        'A61': '< 100 DM', 'A62': '100 - 500 DM', 
        'A63': '500 - 1000 DM', 'A64': '> 1000 DM', 'A65': 'Desconhecido/Sem Poupança'
    },
    "employment": {
        'A71': 'Desempregado', 'A72': '< 1 ano', 
        'A73': '1 - 4 anos', 'A74': '4 - 7 anos', 'A75': '> 7 anos'
    }
}

# Aplicar a tradução no nosso DataFrame Pandas (pdf)
for col_name, mapping in decode_map.items():
    pdf[col_name] = pdf[col_name].replace(mapping)

print("Tradução concluída! Veja como ficou mais legível:")
display(pdf[['checking_status', 'purpose', 'credit_history', 'label']].head())

In [0]:
# Célula 3: Análise da Variável Alvo (Risco)
import seaborn as sns
import matplotlib.pyplot as plt

# Configuração do estilo visual
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))

# Criando o gráfico de contagem
ax = sns.countplot(x='label', data=pdf, palette=['#4CAF50', '#F44336']) # Verde para Bom, Vermelho para Ruim

# Ajustes visuais
plt.title("Distribuição do Risco de Crédito", fontsize=15)
plt.xlabel("Classificação (0=Bom, 1=Ruim)", fontsize=12)
plt.ylabel("Quantidade de Clientes", fontsize=12)
plt.xticks([0, 1], ['Bom Pagador (0)', 'Mau Pagador (1)'])

# Mostrar o gráfico
plt.show()

# Cálculo da porcentagem
bom = pdf['label'].value_counts()[0]
ruim = pdf['label'].value_counts()[1]
print(f"Bons Pagadores: {bom} ({(bom/1000)*100}%)")
print(f"Maus Pagadores: {ruim} ({(ruim/1000)*100}%)")

In [0]:
# Célula 4: Análise Bivariada - Status da Conta vs Risco
plt.figure(figsize=(12, 6))

# Vamos criar um gráfico de barras empilhadas 100% (ou contagem simples com 'hue')
sns.countplot(x='checking_status', hue='label', data=pdf, palette=['#4CAF50', '#F44336'])

plt.title("Risco de Crédito por Saldo em Conta Corrente", fontsize=15)
plt.xlabel("Saldo na Conta", fontsize=12)
plt.ylabel("Quantidade de Clientes", fontsize=12)
plt.legend(title='Risco', labels=['Bom (0)', 'Ruim (1)'])
plt.xticks(rotation=45) # Girar o texto para ler melhor

plt.show()

In [0]:
# Célula 5: Análise Bivariada - Duração vs Risco
plt.figure(figsize=(10, 6))

sns.boxplot(x='label', y='duration', data=pdf, palette=['#4CAF50', '#F44336'])

plt.title("Distribuição da Duração do Empréstimo (Meses) por Risco", fontsize=15)
plt.xlabel("Classificação (0=Bom, 1=Ruim)", fontsize=12)
plt.ylabel("Duração (Meses)", fontsize=12)
plt.xticks([0, 1], ['Bom Pagador', 'Mau Pagador'])

plt.show()